In [1]:
import pandas as pd
import json
import numpy as np
from secret import googlemaps_creds
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Get zillow price data

In [2]:
zillow = pd.read_csv('/home/mrs/housing/zillow/Zip_MedianSoldPrice_AllHomes.csv')
zillow.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06
0,61639,10025,New York,NY,New York,New York,1,175000.0000,NaN,NaN,...,975000.0,1016000.0,962750.0000,1.010000e+06,1079500.0,940500.0000,852750.0000,824746.5,804172.55,883423.95
1,84654,60657,Chicago,IL,Chicago,Cook,2,137911.7647,137900.0,133925.0,...,382500.0,372550.0,363975.0000,3.747250e+05,340150.0,348350.0000,397000.0000,403750.0,395250.00,405875.00
2,61637,10023,New York,NY,New York,New York,3,403823.5294,355025.0,NaN,...,874375.0,869337.5,864287.5000,8.590000e+05,861800.0,850900.0000,853700.0000,887608.0,901530.60,918227.40
3,84616,60614,Chicago,IL,Chicago,Cook,4,168250.0000,191675.0,189250.0,...,456150.0,467650.0,469750.0000,4.551500e+05,464175.0,486425.0000,498750.0000,487000.0,460375.00,445125.00
4,61616,10002,New York,NY,New York,New York,5,NaN,NaN,NaN,...,638100.0,NaN,648076.9231,6.338235e+05,NaN,786153.8462,841764.7059,822000.0,NaN,NaN


In [3]:
zillow = zillow.rename(columns={'RegionName':'zip_code'})
zillow.columns = [c.lower() for c in zillow.columns]
zillow.head()

,regionid,zip_code,city,state,metro,countyname,sizerank,1996-04,1996-05,1996-06,...,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06
0,61639,10025,New York,NY,New York,New York,1,175000.0000,NaN,NaN,...,975000.0,1016000.0,962750.0000,1.010000e+06,1079500.0,940500.0000,852750.0000,824746.5,804172.55,883423.95
1,84654,60657,Chicago,IL,Chicago,Cook,2,137911.7647,137900.0,133925.0,...,382500.0,372550.0,363975.0000,3.747250e+05,340150.0,348350.0000,397000.0000,403750.0,395250.00,405875.00
2,61637,10023,New York,NY,New York,New York,3,403823.5294,355025.0,NaN,...,874375.0,869337.5,864287.5000,8.590000e+05,861800.0,850900.0000,853700.0000,887608.0,901530.60,918227.40
3,84616,60614,Chicago,IL,Chicago,Cook,4,168250.0000,191675.0,189250.0,...,456150.0,467650.0,469750.0000,4.551500e+05,464175.0,486425.0000,498750.0000,487000.0,460375.00,445125.00
4,61616,10002,New York,NY,New York,New York,5,NaN,NaN,NaN,...,638100.0,NaN,648076.9231,6.338235e+05,NaN,786153.8462,841764.7059,822000.0,NaN,NaN


We want to tidy up this dataset - the month columns need to be converted into one column.
Will use pandas.melt to magic our way into that.

In [4]:
cols = zillow.columns.tolist()

In [5]:
zillow = pd.melt(zillow
                 , id_vars=['zip_code', 'city', 'state']
                 , value_vars=cols[8:257]
                 , value_name='median_price'
                 , var_name='month'
                )

In [6]:
zillow.month = pd.DatetimeIndex(zillow.month).to_period('M')

In [7]:
zillow.head()

,zip_code,city,state,month,median_price
0,10025,New York,NY,1996-05,NaN
1,60657,Chicago,IL,1996-05,137900.0
2,10023,New York,NY,1996-05,355025.0
3,60614,Chicago,IL,1996-05,191675.0
4,10002,New York,NY,1996-05,NaN


# Get business data

In [8]:
with open('yelp/yelp_academic_dataset_business.json') as f:
    data = [json.loads(r.strip('\n')) for r in f.readlines()]


In [9]:
biz = pd.DataFrame(data)
biz.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{'Waiter Service': False, 'Good For Groups': T...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{'Wednesday': {'open': '11:00', 'close': '21:0...",40.354327,-79.900706,Mr Hoagie,[],True,7,3.5,PA,business
1,"{'Accepts Credit Cards': True, 'Happy Hour': T...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350553,-79.886814,Clancy's Pub,[],True,5,3.0,PA,business
2,{'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,"1530 Hamilton Rd\nBethel Park, PA 15234",{},40.354115,-80.014660,Cool Springs Golf Center,[],False,5,2.5,PA,business
3,"{'Noise Level': 'average', 'Price Range': 2, '...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"414 Hawkins Ave\nBraddock, PA 15104","{'Wednesday': {'open': '10:00', 'close': '19:0...",40.408830,-79.866211,Emil's Lounge,[],True,26,4.5,PA,business
4,"{'Takes Reservations': True, 'Accepts Credit C...",mYSpR_SLPgUVymYOvTQd_Q,"[Active Life, Golf]",Braddock,"1000 Clubhouse Dr\nBraddock, PA 15104","{'Wednesday': {'open': '11:00', 'close': '20:0...",40.403405,-79.855782,Grand View Golf Club,[],True,3,5.0,PA,business


# Parse categories

Need to parse out the categories so we can pull out the important ones for use in modeling later.
Categories are stored as a list for each business and are obviously not unique across businesses.
So, I take the approach of turning the category series into a list of lists. Then, I pop the first element and extend it by each other element.

But this has many duplicates so I use set to dedupe.

In [10]:
category_list = biz.categories.tolist()
print(len(category_list))
element = category_list.pop()
for c in category_list:
    element.extend(c)
    
categories = sorted(list(set(element)))
len(categories)

85901


1017

In [11]:
categories

['3D Printing',
 'ATV Rentals/Tours',
 'Accessories',
 'Accountants',
 'Active Life',
 'Acupuncture',
 'Addiction Medicine',
 'Adult',
 'Adult Education',
 'Adult Entertainment',
 'Advertising',
 'Afghan',
 'African',
 'Air Duct Cleaning',
 'Airlines',
 'Airport Lounges',
 'Airport Shuttles',
 'Airport Terminals',
 'Airports',
 'Airsoft',
 'Allergists',
 'Alsatian',
 'Amateur Sports Teams',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Anesthesiologists',
 'Animal Shelters',
 'Antiques',
 'Apartments',
 'Appliances',
 'Appliances & Repair',
 'Appraisal Services',
 'Aquarium Services',
 'Aquariums',
 'Arabian',
 'Arcades',
 'Archery',
 'Architects',
 'Argentine',
 'Art Classes',
 'Art Galleries',
 'Art Museums',
 'Art Restoration',
 'Art Schools',
 'Art Supplies',
 'Art Tours',
 'Artificial Turf',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Assisted Living Facilities',
 'Auction Houses',
 'Audiologist',
 'Australian',
 'Austrian',
 'Auto Customiza

I want to flatten this out and make sure I have one row per business. But businesses have multipler categories.
To solve this I'm creating one boolean column per category.

Not the tidiest thing but will make counting by category much simpler.

In [12]:
def bool_column_name(col):
    return 'is_{}'.format(col)

def check_category(category, df):
    df = df.copy()
    column_name = bool_column_name(category)
    df[column_name] = df.categories.apply(lambda x: category in x)
    
    return df[df[column_name]]

In [13]:
def category_flattener(df, categories):
    df = df.copy()
    category_columns = [bool_column_name(cat) for cat in categories]
    category_map = dict(zip(category_columns, categories))
    for cat_col, cat in category_map.items():
        df[cat_col] = df.categories.apply(lambda x: cat in x)
    
    return df[category_columns]

    


In [ ]:
biz = biz.set_index('business_id')

In [ ]:
biz_categories = category_flattener(biz, categories=categories)

In [ ]:
biz_categories

In [ ]:
category_count = biz_categories.sum()

Below, we can count each category. Restaurant is the most common.


In [ ]:
category_count.sort_values(ascending=False).head(20)

What's interesting is that there is hierarchical data here despite not a clear hierarchical structure.

For example, all Bars are Nightlife, but not all Nightlife are Bars (see below)

Furthermore, Restaurant is a giant umbrella for pretty much all food type categories (e.g. Mexican)

For the preliminary analysis I'm going to just use Nightlife, Restaurant, and Food (which is apparently generally grocery stores). However there is likely value in subcategories.

In [ ]:
biz_categories[['is_Nightlife', 'is_Bars']].all(axis=1).sum()

In [ ]:
check_category('Food', biz).head()

# Zip code parsing

This got buried so I'm giving it a header!

Important to note, not all businesses have zip code in their address. Something like 9K are missing it.
I have a method to backfill using reverse geocoding from the google maps API, because all businesses have a latitude and longitude pair, but haven't quite worked it out yet so will just ignore null zips for the preliminary analysis.

In [ ]:
biz['zip_code'] = biz['full_address'].str.extract('([0-9]{5})', expand=True).astype(float)

# Get review data

In [ ]:
reviews=[]
with open('yelp/yelp_academic_dataset_review.json') as f:
   
    lines = f.readlines()
    for line in lines:
        data = json.loads(line.strip('\n'))
        del data['text']
        reviews.append(data)
        

In [ ]:
review_df = (pd.DataFrame(reviews)
               .assign(date=lambda x: pd.to_datetime(x['date']))
              )

In [ ]:
review_df.head()

Here, I am assuming the first review is whent he business opened. Obviously not totally accurate but should be a reasonable proxy. It assumes Yelp is being used reliably in all zip codes under investgigation which may be a faulty assumption.

In [ ]:
review_agg = (review_df.groupby('business_id')
                       .agg({'date':min, 'stars':np.mean})
                       .rename(columns={'date':'first_seen_date', 'stars':'mean_stars'})
             )

In [ ]:
review_agg.head()

In [ ]:
biz_reviews = biz.join(review_agg)

In [ ]:
biz_reviews.head()

In [ ]:
biz_reviews['month'] = pd.DatetimeIndex(biz_reviews.first_seen_date).to_period('M')

In [ ]:
biz_reviews = biz_reviews.join(biz_categories)

In [ ]:
biz_categories.is_Restaurants.head()

In [ ]:
def counter(df, categories):
    df = df.copy().reset_index()
    return df[df[categories]].groupby(['zip_code', 'month']).business_id.count()

In [ ]:
restaurant_counts = counter(biz_reviews, 'is_Restaurants').reset_index()
restaurant_counts.zip_code = restaurant_counts.zip_code.astype(int)

In [ ]:
zillow.month = pd.DatetimeIndex(zillow.month).to_period('M')

In [ ]:
zillow.head()

In [ ]:
combined = pd.merge(zillow, restaurant_counts.reset_index(), how='left', on=['zip_code', 'month'])

In [ ]:
zillow.dtypes

In [ ]:
pitt = combined[combined.city == 'Pittsburgh'].fillna(0)
first_month = pitt[pitt.business_id > 0].month.min()
pitt[pitt.business_id  > 0].head()

In [ ]:
pitt.dtypes

In [ ]:
pitt = pitt[pitt.month >= first_month]
pitt = pitt.set_index(['zip_code', 'month'])
pitt = pitt.sort_index()

In [ ]:
pitt_biz_growth = (pitt
                   .groupby(level=['zip_code', 'month'])
                   .business_id.sum()
                   .groupby(level=['zip_code'])
                   .cumsum()
                  )

In [ ]:
pitt_biz_growth.xs(15206, level='zip_code').tail()

In [ ]:
g = sns.FacetGrid(data=pitt_biz_growth.reset_index(), col='zip_code', col_wrap=3)
g = g.map(sns.pointplot, 'month', 'business_id')


In [ ]:
pitt.groupby(level='zip_code').median_price.median()

In [ ]:
pitt_prices.month.min()

In [ ]:
pitt_prices = pitt.reset_index()
pitt_prices = pitt_prices[pitt_prices.month > first_month]
s = sns.FacetGrid(data=pitt_prices.reset_index(), col='zip_code', col_wrap=3)
s = s.map(sns.pointplot, 'month', 'median_price')